<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/Test/REO_P_C_ch_50_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
* Task : REO
* subject : 50
* 28 channel(P,C): 

{  C5, C3, C1, Cz, C2, C4,C6, Cp5, Cp3, Cp1, Cpz, Cp2, Cp4, Cp6, P7, P5, P3, P1,
Pz, P2, P4, P6, P8, Po7, Po3, Poz, Po4, Po8} 

(index = 7,8,9,10,11,12,13,14,15,16,17,18,19,20,46,47,48,49,50,51,52,53,54,55,56,57,58,59)

* model:

layers: 3 conv2D(3,3) & maxpooling(2,2) - 1 Dense(512)
epoch: 30
batch size: 20

'''



import matplotlib

from matplotlib import pyplot as plt
import numpy as np

import math

import keras
from keras.utils import to_categorical

import random
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

Using TensorFlow backend.



* Task : REO
* subject : 50
* 26 channel(F): 

{ Fc5, Fc3, Fc1, Fcz, Fc2, Fc4, Fc6, Fp1, Fpz, Fp2, Af7,
Af3, Afz, Af4, Af8, F7, F5, F3 F1, Fz, F2, F4, F6, F8, Ft7, Ft8} 

(index = 0,1,2,3,4,5,6, 21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39)

* model:

layers: 3 conv2D(3,3) & maxpooling(2,2) - 1 Dense(512)
epoch: 30
batch size: 20






In [2]:
subject_number=109
task_index = [1] # task: REC
task_number = len(task_index)


# 28 channels (P,C)
list_channel = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,46,47,48,49,50,51,52,53,54,55,56,57,58,59]
ch_number = len(list_channel)

task_time = 60

sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 4 #step len
window_len= 20

task_number

1

In [0]:
# load dataset in array
list_raw_fnames = [[0]*task_number]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,task_index)

list_rawdata = np.zeros((subject_number,task_number), dtype='object')

for i in range(subject_number):
    for j in range(task_number):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)


In [0]:
# img for each one
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_start_index=subject_img_number - math.floor(subject_img_number * 0.15) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index
train_number

In [0]:
train_img = np.zeros((train_number*subject_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number, ch_number, window_len, 160), dtype = float)
test_label =[]


train_img.shape

In [0]:

def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ))
  return ch

In [0]:
list_rawdataTest=list_rawdata.copy()


for s in range(subject_number):
    ch_max =[]
    ch_min =[]
    DataChannelsNormal=np.zeros((ch_number,total_sampel_number),dtype = float)
    i =0
    for ch in list_channel: 
      DataChannelsRaw, times =list_rawdata[s][0][ch,:total_sampel_number]
      
      # make a copy
      DataChannelsRaw_t=DataChannelsRaw.copy()
      times_t=times.copy()
      
      ch_max = np.append(ch_max, max(DataChannelsRaw_t[0])) # max for each cannel
      ch_min = np.append(ch_min, min(DataChannelsRaw_t[0])) # min for each cannel
      
      # Normalize Channels
      DataChannelsNormal[i]=normalize_channel_data(DataChannelsRaw_t, i, ch_min, ch_max)
      i=i+1
      
    for j in range (subject_img_number):
        for i in range(ch_number):
            for z in range (window_len):
              
              ExtractedData = DataChannelsNormal[i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]
              
              if j <test_start_index :
                    train_img[s*train_number + j][i][z] = ExtractedData
                    if z==window_len-1 :
                      train_img[s*train_number + j][i][z+1] = s
                      
                      
              else:
                    test_img[s*test_number + j - test_start_index][i][z] = ExtractedData
                        
                    
              
        if j >=test_start_index :
          test_label = np.append(test_label, (s))

In [0]:
DataChannelsRaw_t[0]
DataChannelsNormal.shape

In [0]:
#shaffle train img array
train_img_shuffle = train_img.copy()

np.random.shuffle(train_img_shuffle)

In [0]:
train_img_2 = np.zeros((train_number*subject_number*task_number, ch_number, window_len, 160), dtype = float)

# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)



In [0]:
train_img_2.shape

In [0]:

train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [0]:
validation_index = math.floor( len(train_img_2) * 0.2)
validation_index

# valid & train
x_train =train_img_2[validation_index:]
y_train =train_label[validation_index:]

x_valid =train_img_2[:validation_index]
y_valid =train_label[:validation_index]

x_valid.shape

In [0]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', padding='same', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(256, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

In [15]:
from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [0]:
history = model.fit(
        x_train,
        y_train,
        epochs = 30,
        batch_size = 20,
        validation_data = (x_valid, y_valid)
)

In [0]:
model.evaluate(test_img, test_label)


In [0]:
import matplotlib.pyplot as plt

history_dict = history.history 

loss_values = history_dict ['loss'] 

val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)



plt.plot(epochs, loss_values, 'bo', label='Training loss') 

plt.plot(epochs, val_loss_values, 'b', label='Validation loss') 

plt.title('Training and validation loss') 

plt.xlabel('Epochs') 

plt.ylabel('Loss') 

plt.legend()

plt.show()

In [0]:
history_dict = history.history 

acc_values = history_dict ['acc'] 

val_acc_values = history_dict['val_acc']



plt.plot(epochs, acc_values, 'bo', label='Training acc') 

plt.plot(epochs, val_acc_values, 'b', label='Validation acc') 

plt.title('Training and validation acc') 

plt.xlabel('Epochs') 

plt.ylabel('acc') 

plt.legend()

plt.show()